# Data Mining Project 3: Analysis of the global video games market

## Introduction

With the release of the first video game in early 1970s, a ping pong game by Arcade, the video game industry has grown considerably. According to a [report](https://www.zionmarketresearch.com/report/cloud-gaming-market) from the Zion Market Research, the global gaming market was valued at 802 USD million in 2017 and is expected to grow for the following years. 

This project explores the growth of the global gaming industry. This sector has become extremely dynamic and complex. Therefore, it is relevant for publishers and developers of video games to understand the major changes in the gaming industry over the last decades and can we expect for the upcoming years.

To answer these questions, this project explores the following topics:

* Most popular video games by genre
* Most popular video games by region
* Most popular consoles


---
**Github and Zenodo**

The project can be found in [GitHub account](https://github.com/svaldess/DATS6103-Project3) and as a [GitHub Page](https://svaldess.github.io)

It can also be ofund in [Zenodo](https://zenodo.org/record/3571479#.XfGjEOhKhPY)

---

## About the source and the dataset

The data of this project was extracted from [VGCchartz](http://www.vgchartz.com/gamedb/), a video game sales tracking website. It has information about the consoles, developers, scores, sales, genre, and the year when the video games were released. VGChartz has a vast information of video games since the 1970s. 

The information was scrapped from VGChartz website and `BeutifulSoup` was used for parsing the page. The code used for extracting the information is presented in a separate Jupyter Notebook, called **Project3_WebScrapper**. After the information was successfully extracted, it was stored in a csv file, called "vgames.csv", that was used for reading and manipulating the data.

## Explore and clean dataframe

In [1]:
#import all required libraries
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
#in case chart_studio is not installed, please install it with the following code:
#pip install chart_studio

In [2]:
#Connect to my chart_studio account
import chart_studio.plotly as py
py.sign_in('svaldes.salas', 'owvxCCaHjEJC81iQ5p8B')

In [4]:
# Create directory for saving images
import os
if not os.path.exists("images"):
    os.mkdir("images")

In [5]:
#store the csv file 'vgames.csv' into dataframe games
games = pd.read_csv("vgames.csv")

The csv file was succesfully converted into a data frame. Let's take a look at the first rows.

In [6]:
games[:5]

,Unnamed: 0,Genre,Name,Console,Publisher,Developer,Critic_score,User_score,Total_sales,America_sales,PAL_sales,Japan_sales,Other_sales,Release_date,Update_date
0,0,Action,Final Fight,SNES,Capcom,Capcom,10.0,NaN,1.56m,0.67m,0.17m,0.69m,0.03m,10th Nov 91,NaN
1,1,Action,Red Dead Redemption: Undead Nightmare,PS3,Rockstar Games,Rockstar San Diego,10.0,NaN,1.18m,0.47m,0.45m,0.06m,0.19m,23rd Nov 10,NaN
2,2,Action,Red Dead Redemption: Undead Nightmare,X360,Rockstar Games,Rockstar San Diego,10.0,NaN,1.11m,0.61m,0.38m,0.02m,0.10m,23rd Nov 10,NaN
3,3,Action,Grand Theft Auto IV,PS3,Rockstar Games,Rockstar North,10.0,9.0,10.57m,4.79m,3.73m,0.44m,1.62m,29th Apr 08,NaN
4,4,Action,Grand Theft Auto IV,X360,Rockstar Games,Rockstar North,10.0,NaN,11.09m,6.80m,3.11m,0.14m,1.04m,29th Apr 08,NaN


In [7]:
#Check types of data in each column
games.dtypes

Unnamed: 0         int64
Genre             object
Name              object
Console           object
Publisher         object
Developer         object
Critic_score     float64
User_score       float64
Total_sales       object
America_sales     object
PAL_sales         object
Japan_sales       object
Other_sales       object
Release_date      object
Update_date       object
dtype: object

In [8]:
#The dataframe has 15 columns and more than 50k rows
games.shape

(56897, 15)

**Drop unnecessary columns.**

In [9]:
#drop first column 'unnamed' and column 'num'
games = games.drop(games.columns[0], axis=1)
games.head()

,Genre,Name,Console,Publisher,Developer,Critic_score,User_score,Total_sales,America_sales,PAL_sales,Japan_sales,Other_sales,Release_date,Update_date
0,Action,Final Fight,SNES,Capcom,Capcom,10.0,NaN,1.56m,0.67m,0.17m,0.69m,0.03m,10th Nov 91,NaN
1,Action,Red Dead Redemption: Undead Nightmare,PS3,Rockstar Games,Rockstar San Diego,10.0,NaN,1.18m,0.47m,0.45m,0.06m,0.19m,23rd Nov 10,NaN
2,Action,Red Dead Redemption: Undead Nightmare,X360,Rockstar Games,Rockstar San Diego,10.0,NaN,1.11m,0.61m,0.38m,0.02m,0.10m,23rd Nov 10,NaN
3,Action,Grand Theft Auto IV,PS3,Rockstar Games,Rockstar North,10.0,9.0,10.57m,4.79m,3.73m,0.44m,1.62m,29th Apr 08,NaN
4,Action,Grand Theft Auto IV,X360,Rockstar Games,Rockstar North,10.0,NaN,11.09m,6.80m,3.11m,0.14m,1.04m,29th Apr 08,NaN


**Check information about release date and update date.**

By exploring _Release_date_ and _Update_date_ columns we can see that there are some missing data, especially in the update column.

In [10]:
#There are more missing values in Update_date column
print(games.Release_date.isna().sum())
print(games.Update_date.isna().sum())

2955
46216


Let's create a third column using, primarily, values from _Release_date_. In case the values are zero value, let's use the value from _Update_date_. This will be used in order to reduce the sparse data associated with date.

In [11]:
#Create a function to fetch data from 2 columns and create a 3rd column
def converter(primcol, secol):
    #Turn NAN's into 0 in primary column
    games[primcol] = games[primcol].fillna(0)
    #Create objects
    primary = games[primcol]
    secondary = games[secol].fillna(0)
    #Create third column
    third = np.where(games[primcol]!=0, primary, secondary)
    return third

In [13]:
#Before converting, 'Release_date' from row 56882 is equal to NAN
games.loc[56882,'Release_date']

nan

In [14]:
alldate = converter(primcol='Release_date',secol='Update_date')
games = games.drop(['Release_date','Update_date'],axis=1)

In [15]:
#After converting, 'Date' from row 56882 is different from zero (corresponds to the value from Update_date column)
games['Date'] = alldate
games.loc[56882,'Date']

'01st Mar 19'

In [16]:
#1,282 values from Update_date were use to filled the zero values in Date column
count=0
for row in games.Date:
    if row ==0:
        count+=1
print(2955-count)

1282


In [17]:
games[:3][['Date']]

,Date
0,10th Nov 91
1,23rd Nov 10
2,23rd Nov 10


Now, let's extract information from column "Date" and create three columns about 'day', 'month', and 'year'. 

In [18]:
#Create a function for extracting information from column 'Release_date'
def fetch_date(string, date):
    if date=='month':
        try:
            month = string.split(sep=" ")[1].strip()
            return month
        except: 
            return 'NaN'
    else:
        try:
            day = string.split(sep=" ")[0][:2]
            return day
        except:
            return 'NaN'

In [19]:
#Extract information abput month and create a column called 'month' 
games['month'] = games['Date'].apply(lambda x: fetch_date(x, date='month'))

#We can also extract the day of the date
games['day'] = games['Date'].apply(lambda x: fetch_date(x, date='day'))

In [20]:
#convert Date into datetime
games['Date'] = pd.to_datetime(games['Date'])

In [21]:
#We can extract the information about the year from 'Date' column
games['year'] = games['Date'].map(lambda x: x.year)

In [22]:
#the function worked
games[:3][['Date','month','day','year']]

,Date,month,day,year
0,1991-11-10,Nov,10,1991
1,2010-11-23,Nov,23,2010
2,2010-11-23,Nov,23,2010


In [23]:
#We can see that the column 'Date' is type datetime
games.dtypes

Genre                    object
Name                     object
Console                  object
Publisher                object
Developer                object
Critic_score            float64
User_score              float64
Total_sales              object
America_sales            object
PAL_sales                object
Japan_sales              object
Other_sales              object
Date             datetime64[ns]
month                    object
day                      object
year                      int64
dtype: object

In [24]:
#Let's check that there are no missing values in year and month columns
print(games['year'].isna().sum())
print(games['month'].isna().sum())

0
0


**Check information about sales**

First, let's convert all columns about sales into numeric. In order to do this, let's extract the float from the string, which has the following format: "#.##m" The number is extracted with *str.extract()* function and with using regex.

In [26]:
games['Total_sales'] = games.Total_sales.str.extract('(\d+\.\d+)')
games['America_sales'] = games.America_sales.str.extract('(\d+\.\d+)')
games['PAL_sales'] = games.PAL_sales.str.extract('(\d+\.\d+)')
games['Japan_sales'] = games.Japan_sales.str.extract('(\d+\.\d+)')
games['Other_sales'] = games.Other_sales.str.extract('(\d+\.\d+)')

In [27]:
#The number was succesfully extracted.
games[:3][['Total_sales','America_sales','PAL_sales','Japan_sales','Other_sales']]

,Total_sales,America_sales,PAL_sales,Japan_sales,Other_sales
0,1.56,0.67,0.17,0.69,0.03
1,1.18,0.47,0.45,0.06,0.19
2,1.11,0.61,0.38,0.02,0.10


In [28]:
#Create a function that converts all column sales into numeric
sales = ['Total_sales','America_sales','PAL_sales','Japan_sales','Other_sales']
for sale in sales:
    games[sale] = pd.to_numeric(games[sale], errors='coerce')

#Column sales were converted into numeric
games.dtypes

Genre                    object
Name                     object
Console                  object
Publisher                object
Developer                object
Critic_score            float64
User_score              float64
Total_sales             float64
America_sales           float64
PAL_sales               float64
Japan_sales             float64
Other_sales             float64
Date             datetime64[ns]
month                    object
day                      object
year                      int64
dtype: object

Now let's explore the data in the year 2019. 

In [29]:
#Check values in column total_sales
year19 = games.loc[games.year==2019]
year19 = year19.loc[:,['Total_sales','America_sales','PAL_sales','Japan_sales',
                       'Other_sales','year','month']].groupby(['year','month']).sum()
year19

Total_sales  America_sales  PAL_sales  Japan_sales  Other_sales
year month                                                                 
2019 Apr           0.11            0.0       0.06         0.04         0.01
     Aug           0.02            0.0       0.00         0.02         0.00
     Dec           0.28            0.0       0.04         0.23         0.00
     Feb           1.28            0.0       0.00         1.28         0.00
     Jan           0.09            0.0       0.00         0.09         0.00
     Jul           0.29            0.0       0.00         0.29         0.00
     Jun           0.32            0.0       0.00         0.32         0.00
     Mar           0.17            0.0       0.02         0.15         0.00
     May           0.01            0.0       0.00         0.01         0.00
     Nov           0.03            0.0       0.00         0.03         0.00
     Oct           0.00            0.0       0.00         0.00         0.00
     Sep           0.13            0.0       0.00         0.13         0.00

By checking the values in 2019, we can see that all values dropped considerably. This may be because the data from the website has not been updated for this year. For this reason, it is better to analyse tha dataset up to 2018.

In [30]:
#Clean years and just use 1980 to 2018
games = games.loc[(games.year >= 1980)&(games.year < 2019)]
len(games)

52361

**Check information about critic and user score.**

Both _Critic_score_ and _User_score_ have they have many zero values. 

In [31]:
#Number of NAN's in critic_score and user_score is large
print(games.Critic_score.isna().sum())
print(games.User_score.isna().sum())

45852
52007


To reduce this, we'll create only one column about scores with information about critic and user scores. If _Critic_score_ is different from zero, use the data; if _critic_score_ is zero, use the data from _User_score_. Use all information in a column called _score_column_.

In [32]:
allscore = converter(primcol='Critic_score',secol='User_score')
games = games.drop(['Critic_score','User_score'],axis=1)

In [33]:
games['Scores'] = allscore
games[:3][['Scores']]

,Scores
0,10.0
1,10.0
2,10.0


## Analysis of the video games market by genre

### Releases by year, month and day

In [34]:
#Create a counting variable for 'genre'
games['count'] = 1

In [36]:
#Let's check the total number of video games released by year
genre_per_year = games.loc[:,['year','Genre','count']]
total_per_year = genre_per_year.groupby(['year']).sum().reset_index()
fig = go.Figure(data=go.Scatter(x=total_per_year['year'], y=total_per_year['count'], mode='lines+markers'))
fig.update_layout(title="Number of video games released by year, 1980-2018")
py.iplot(fig)

In [37]:
#Save image
fig.write_image("images/fig1.png")

With the graph above we can see that there has been an increasing trend from the 1980s to the end of 2010s. Since then, the trend has not been clear.

In [39]:
#Let's check the total number of video games released by month
genre_per_month = games.loc[:,['month','Genre','count']]
total_per_month = genre_per_month.groupby(['month']).sum()
#re order the index of the data frame
total_per_month = total_per_month.reindex(["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"])

#color bars with the highest number of releases 
colors = ['lightslategray',] *12
colors[0] = 'crimson'
colors[8] = 'crimson'
colors[9] = 'crimson'
colors[10] = 'crimson'
#plot 
fig2 = go.Figure(data=[go.Bar(
    x=total_per_month.index,
    y=total_per_month['count'],
    marker_color=colors 
)])
fig2.update_layout(title_text='Number of video games released by month, 1980-2018')

py.iplot(fig2)

In [40]:
#Save image
fig2.write_image("images/fig2.png")

For the period 1980 to 2018, January, November, October, and September are the months when most videogames are released (in that order)

In [43]:
#Let's check the total number of video games released by day
genre_per_day = games.loc[:,['day','Genre','count']]
total_per_day = genre_per_day.groupby(['day']).sum().reset_index()
#plot total count per day
fig3 = px.bar(total_per_day, x="count", y="day", orientation='h')
fig3.update_layout(title="Number of video games released by day of the month, 1980-2018")
py.iplot(fig3)

In [44]:
#Save image
fig3.write_image("images/fig3.png")

For the period 1980 to 2018, video games have mostly been released the 1st day of the month.

### Releases by genre

In [45]:
#Let's get the top genres of all time 
grouped_genre = games.groupby('Genre').sum()
totals = grouped_genre[['count','Total_sales']].reset_index()
Top_genreAlltime = totals.loc[:,['Genre','count']].set_index('Genre').sort_values(by='count', ascending=False)
#Create a table and color those rows depending on their value
myvalues = Top_genreAlltime['count'].unique().tolist()
mygenres = Top_genreAlltime.index.tolist()
table_trace=dict(type = 'table',
                 columnwidth= [20]+[10],
                 columnorder=[0, 1],
                 header = dict(height = 50,
                               values = [['<b>Genre</b>'], ['<b>Total</b>']],
                               line = dict(color='rgb(50,50,50)'),
                               align = ['left']*2,
                               font = dict(color=['rgb(45,45,45)']*2, size=14),
                               fill = dict( color = 'rgb(222, 222, 222)' )
                              ),
                 cells = dict(values = [mygenres, myvalues],
                              line = dict(color='#506784'),
                              align = ['left']*5,
                              font = dict(color=['rgb(40,40,40)']*2, size=12),
                              height = 30,
                              fill = dict(color=['rgb(245,245,245)',
                                                ['rgb(121, 96, 214)' if value>=4000 else 
                                                 'rgb(194, 183, 232)' if value<2000 else
                                                 'rgb(145, 128, 209)' for value in myvalues] ]
                                         )
                             )
                  )

layout = dict(width=400, height=600, autosize=False, 
              title='Count of video games by genre, 1980-2018', showlegend=False)
fig4= dict(data=[table_trace], layout=layout)
py.iplot(fig4, filename='count-videogames')

With the exception of **Misc** (which we assume is miscellaneous), **Action** is the genre with the most video games releases, followed by **Adventure**, **Sports**, **Role-playing**, and **Shooter**. Each of these genres have accumulated over 4,000 video games in the last 40 years. 

The total number of video games does not tell us much about those genres that are relevant for the industry. For this reason, it is more relevant to analyse which represent the most profitable genres in the gaming industry. Let's compare sales by genre over time.

In [47]:
#Let's check total number of released video games and sales
fig5 = px.bar(totals.sort_values(by='count'), x='Genre', y='Total_sales',
             color='count',
             labels={'Total_sales':'Total sales',
                    'count':'Total count per genre'})
fig5.update_layout(title_text='Total sales and total number of video games by genre, 1980-2018')
py.iplot(fig5)

In [48]:
#Save image
fig5.write_image("images/fig5.png")

With the exception of Misc (bright yellow), from 1980 to 2018, the **genre** with the largest count are:
1. Action (yellow)
2. Sports
3. Adventure 
4. Shooter 
5. Role-playing (dark pink)

However, if we take a closer look, those genres that represent the largest **total sales** during the same period are:
1. Sports
2. Action
3. Shooter
4. Role-playing 
5. Platform

Many of those genres in blue might be genres that have been recently created. For this reason, it is a good idea to compare the most profitable genres of the gaming industry in a shorter period, for example, 2008 and 2018. 

### Composition of the video game market in 1998 and 2018

In [50]:
# Get percentage distribution of total sales by genre in 1998
sales_per_year = games.loc[:,['year','Genre','Total_sales']]
comparedsales = sales_per_year.loc[sales_per_year.year == 1998].drop('year',axis=1)
sales98 = comparedsales.groupby('Genre').sum()
sales98['%'] = sales98['Total_sales'].apply(lambda x:x/sales98.sum()*100)
sales98 = sales98.sort_values(by='%',ascending=False)[:10]

# Get percentage distribution of total sales by genre in 2018
comparedsales18 = sales_per_year.loc[sales_per_year.year == 2018].drop('year',axis=1)
sales18 = comparedsales18.groupby('Genre').sum()
sales18['%'] = sales18['Total_sales'].apply(lambda x:x/sales18.sum()*100)
sales18 = sales18.sort_values(by='%',ascending=False)[:10]

# Define a function to create labels and values of the pie charts.
def ValuesLabels(df, column='%'):
    t = 0
    values = []
    labels = []
    for index, row in df.iterrows():
        v = row[column]
        t += v
        values.append(v)
        labels.append(index)
    values.append(100-t)
    labels.append('Others')
    return values, labels

#Get values and labels for 1998 and 2018
values98 = ValuesLabels(sales98)[0]
labels98 = ValuesLabels(sales98)[1]
values18 = ValuesLabels(sales18)[0]
labels18 = ValuesLabels(sales18)[1]

# Create subplots: use 'domain' type for Pie subplot
fig6 = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig6.add_trace(go.Pie(labels=labels98, values=values98, name="1998"),
              1, 1)
fig6.add_trace(go.Pie(labels=labels18, values=values18, name="2018"),
              1, 2)

fig6.update_traces(hole=.4)
fig6.update_layout(
    title_text="Distribution of video games by genre as a share of total sales in 1998 and 2018",
    annotations=[dict(text='1998', x=0.19, y=0.5, font_size=20, showarrow=False),
                 dict(text='2018', x=0.81, y=0.5, font_size=20, showarrow=False)])
py.iplot(fig6)

In [51]:
#Save image
fig6.write_image("images/fig6.png")

The video game market has completely changed in the last 10 years. In 1998, Role-Playing, Sports, and Racing represented the most profitable genres. In 2018 Action-Adventure (one of the newest genres) replaced Sports as the most profitable genre, which took the second place, followed by Action.

However, it is worth noticing that during this 10-year period, Sports remains in the top 3 of the most profitable genres.

### Total sales over time

In [52]:
#Let's check total sales
sales_grouped = games.loc[:,['Genre','Total_sales','America_sales','PAL_sales','Japan_sales','Other_sales','year']]
sales_per_genre = sales_grouped.groupby(['Genre','year']).sum().reset_index()
sales_per_year = sales_grouped.groupby(['year']).sum()
#Plot
fig7 = px.scatter(x=sales_per_genre.year, y=sales_per_genre.Total_sales, color=sales_per_genre.Genre)
fig7.update_layout(
    title="Total sales of video games per year and per genre from 1980 to 2018",
    xaxis_title="Year",
    yaxis_title="Sales (million USD)")
py.iplot(fig7)

In [53]:
#Save image
fig7.write_image("images/fig7.png")

After 2000, the number of genres not only multiplied; also their sales increased drastically. Also notice that sales of  Sports, Action and Shooter soared in the last 20 years. In the case of Sports, sales peaked especially between 2005 and 2010.

In [55]:
dec80 = sales_per_genre[sales_per_genre.year<1990]['Total_sales']
dec90 = sales_per_genre[(sales_per_genre.year>=1990)&(sales_per_genre.year<2000)]['Total_sales']
dec00 = sales_per_genre[(sales_per_genre.year>=2000)&(sales_per_genre.year<2010)]['Total_sales']
dec10 = sales_per_genre[sales_per_genre.year>=2010]['Total_sales']

x_data = ['1980s', '1990s','2000s', '2010s']
y_data = [dec80, dec90, dec00, dec10]
colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)','rgba(255, 65, 54, 0.5)']
#plot
fig8 = go.Figure()
for xd, yd, cls in zip(x_data, y_data, colors):
        fig8.add_trace(go.Box(
            y=yd,
            name=xd,
            boxpoints='all',
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker_size=2,
            line_width=1)
        )
fig8.update_layout(title='Total sales of video games in different decades',
                 yaxis_title="Sales (million USD)")      
py.iplot(fig8)

In [56]:
#Save image
fig8.write_image("images/fig8.png")

The range of sales has increased dramatically. Whereas in the 1980s, the median value of total sales was 0.74 million dollars, in the 2000s it was 24.75 million dollars (this is 33 times larger than in 1980s). 

Let's take a closer look to those top 5 video games that dramatically increased their sales after 2005.

In [57]:
#Get trends of sales of those genres that have the largest sales after 2000
total_sales_genre = sales_per_genre.loc[:,['year','Genre','Total_sales']]
total_sales_pivot = pd.pivot_table(total_sales_genre, values='Total_sales',index='year',columns='Genre')
reduced_genre = total_sales_pivot.reset_index().fillna(0)
reduced_genre = reduced_genre.loc[:,['year','Action','Adventure','Shooter',
                                    'Sports','Role-Playing']]
reduced_genre
colnames=list(reduced_genre.columns[1:])
fig9 = go.Figure()
for name in colnames:
    fig9 = fig9.add_trace(go.Scatter(x=reduced_genre['year'], y=reduced_genre[name], mode='lines', name=name))
fig9.update_layout(
    title="Total sales of most popular video games by genre from 1980 to 2018",
    xaxis_title="Year",
    yaxis_title="Sales (million USD)")
py.iplot(fig9)

In [58]:
#Save image
fig9.write_image("images/fig9.png")

### Total video games by console

In [59]:
#Get total number of video games by console
total_console = games.loc[:,['Console','count','year']]
total_console = total_console.loc[total_console['year']>2000]
count_console = total_console.groupby('Console').sum()[['count']].reset_index()
largevalues = count_console[count_console['count']>500].sort_values('count')
#plot
fig10 = px.bar(largevalues, x="count", y="Console", orientation='h')
fig10.update_layout(title="Total video games released by console from 2000 to 2018")
py.iplot(fig10)

In [60]:
#Save image
fig10.write_image("images/fig10.png")

PC console has the largest number of total video games released. His closest competitor is play station (PS2). However, PC represents only one console, whereas Play Station, Xbox, among others, have different consoles. For this reason, we can merge the consoles of the most popular brands in order to see if they are direct competitors of PC. 

### Competition among most popular consoles

In [67]:
#compare PC with all consoles from play station, xbox and nintendo
PcPs_all = games[(games['Console']=='PC')|(games['Console']=='PS')|(games['Console']=='PS2')|(games['Console']=='PS3')|
                 (games['Console']=='PS4')|(games['Console']=='X360')|(games['Console']=='XB')|
                (games['Console']=='XOne')|(games['Console']=='SNES')|(games['Console']=='Wii')|(games['Console']=='N64')]
#Let's check only those years from 1990 onwards
PcPs_all = PcPs_all[PcPs_all['year']>=1990]
compare_count = PcPs_all.loc[:,['year','Console','count']]

In [68]:
#Create a function to replace names of consoles
def replacer(df):
    df['Names'] = df['Console'].str.replace('PS.','PS',regex=True).str.replace('X.+','XBox',regex=True).str.replace('Wii','Nin',regex=True).str.replace('N64','Nin',regex=True).str.replace('SNES','Nin',regex=True)
    return df

In [69]:
#Check if replacer function works = it works
compare_count = replacer(compare_count)
compare_count[:5]

,year,Console,count,Names
0,1991,SNES,1,Nin
1,2010,PS3,1,PS
2,2010,X360,1,XBox
3,2008,PS3,1,PS
4,2008,X360,1,XBox


In [70]:
compare_count = compare_count.pivot_table(index='year',values='count',columns='Names', aggfunc=np.sum)
fig11 = go.Figure()
fig11 = fig11.add_trace(go.Scatter(x=compare_count.index, y=compare_count['PC'], mode='lines', name='PC', fill='tozeroy'))
fig11 = fig11.add_trace(go.Scatter(x=compare_count.index, y=compare_count['PS'], mode='lines', name='PS', fill='tozeroy'))
fig11 = fig11.add_trace(go.Scatter(x=compare_count.index, y=compare_count['XBox'], mode='lines', name='XBox', fill='tozeroy'))
fig11 = fig11.add_trace(go.Scatter(x=compare_count.index, y=compare_count['Nin'], mode='lines', name='Nintendo', fill='tozeroy'))
fig11.update_layout(title="Number of video games released for PC, Play Station, XBox and Nintendo consoles, 1990-2018")
py.iplot(fig11)

In [71]:
#Save image
fig11.write_image("images/fig11.png")

Play Station has been a strong competitor of PC, especially between the period of 1995 to 2010. After 2010, however, PC became the brand with the largest number of video games. 

In [92]:
#Let's check total sales of each console
PcPs = PcPs_all.loc[:,['year','Console','Genre','count','Total_sales','America_sales']]
PcPs2000 = PcPs.groupby(['year','Console']).sum().reset_index()
PcPspivot = pd.pivot_table(PcPs2000, values='Total_sales',index='year',columns='Console').fillna(0)

#conames = PcPs2000['Console'].unique().tolist()
conames = ['SNES', 'PS', 'N64', 'PS2', 'XB', 'X360', 'PS3', 'Wii', 'PS4', 'PC']
fig12 = go.Figure()
for name in conames:
    fig12 = fig12.add_trace(go.Scatter(x=PcPspivot.index, y=PcPspivot[name], mode='lines', name=name, fill='tozeroy'))
fig12.update_layout(title="Total sales of released video games by console, 1990-2018")
py.iplot(fig12)

In [93]:
#Save image
fig12.write_image("images/fig12.png")

* Nintendo was the dominant player between 1990 to 1995, with the console Super Nintendo. It became popular again with the introduction of Wii in 2006.
* From 1995 to 2005, Play Station became the most profitable brand with the consoles PS and PS2. 
* For the next 10 years (2005 to 2015), the gaming industry became more competitive: Nintendo once more became the most profitable company with Wii console; but it had strong competitors: Xbox 360 and PS3. We can name this the golden age of video games.
* During all this years, even though PC has the largest number of video games released, its sales constantly represent just a small share of total sales.

### Most popular video games and consoles by region

In [76]:
#Check sales by regions after 2000
regions = sales_per_year.reset_index()
regions = regions.loc[regions.year>=2000]
#plot
fig13 = go.Figure(data=[
    go.Bar(name='North America', x=regions['year'], y=regions['America_sales'], marker_color='#0c6a9c'),
    go.Bar(name='Japan', x=regions['year'], y=regions['Japan_sales'], marker_color='#1393d6'),
    go.Bar(name='PAL region', x=regions['year'], y=regions['PAL_sales'], marker_color='#0cc4c1'),
    go.Bar(name='Other', x=regions['year'], y=regions['Other_sales'], marker_color='#0cc48a')],
    layout_title_text='Total sales of video games by regions, 2000-2018')
fig13.update_layout(barmode='stack')
fig13.update_xaxes(title_text='year')
fig13.update_yaxes(title_text='sales (million USD)')
py.iplot(fig13)

In [77]:
#Save image
fig13.write_image("images/fig13.png")

North America represents the region with the largest sales of video games, followed by PAL region (which consists in some regions in Latin America, Middle East and Africa). Also, 2008 represented the most important year in terms of sales. 

In [81]:
#Let's check popularity of top consoles by region
PcPs_sales = PcPs_all.loc[:,['Console','Total_sales','America_sales','PAL_sales','Japan_sales','Other_sales']]
PcPs_sales = PcPs_sales.groupby(['Console']).sum()
PcPs_perc = PcPs_sales.apply(lambda x:x/PcPs_sales['Total_sales']*100)
#Re order rows so that consoles are grouped by companies
PcPs_perc = PcPs_perc.reindex(['PC','PS','PS2','PS3','PS4','SNES','N64','Wii','X360','XB','XOne'])

#plot percentage changes
fig14 = go.Figure(data=[
    go.Bar(name='North America', x=PcPs_perc.index, y=PcPs_perc['America_sales'], marker_color='#4d54e8'),
    go.Bar(name='Japan', x=PcPs_perc.index, y=PcPs_perc['Japan_sales'], marker_color='#040b94'),
    go.Bar(name='PAL region', x=PcPs_perc.index, y=PcPs_perc['PAL_sales'], marker_color='#c310cc'),
    go.Bar(name='Other', x=PcPs_perc.index, y=PcPs_perc['Other_sales'], marker_color='#d4d420')])

fig14.update_layout(barmode='stack')
fig14.update_layout(title="Percentage distribution of sales by console and by region, 1990-2018")
py.iplot(fig14)

In [82]:
#Save image
fig14.write_image("images/fig14.png")

* PC sales have been dominant in PAL region.
* Over time, play station has gained strength in PAL region: PS4 has the largest sales in this region.
* Super Nintendo was dominant in Japan, but Wii is more popular in North America.
* Finally, Xbox remains popular in North America.

### Top publishers and brands

In [83]:
#Check top 20 publishers since 2000
all_publishers = games.loc[:,['Publisher','Total_sales','year','count','Console']]
pub2000 = all_publishers[all_publishers['year']>2000]
pub2000 = pub2000.loc[pub2000['Publisher']!='Unknown']
pub = pub2000.groupby('Publisher').sum()[['count']].sort_values(by='count',ascending=False)

#Create a table
top20 = pub[:20]
val = [x for x in range(1,21)]

fig15 = go.Figure(data=[go.Table(header=dict(values=['Ranking','Top Publisher 2000-2018', 'Total number of video games']),
                 cells=dict(values=[val, top20.index, top20['count']]))
                     ])
py.iplot(fig15)

Ubisoft, Activision and Microsoft remain the publishers with the largest number of video games released between 2000 and 2018. Both Ubisoft and Activision have mostly released video games for PC. 

In [84]:
ubisoft = pub2000[pub2000['Publisher']=='Ubisoft']
ubisoft['Console'].value_counts()[:3]

PC     265
DS     192
Wii    127
Name: Console, dtype: int64

In [85]:
activision = pub2000[pub2000['Publisher']=='Activision']
activision['Console'].value_counts()[:3]

PC      188
X360    148
DS      124
Name: Console, dtype: int64

In [86]:
#Check total sales of publisher for each year
publisher = pub2000.groupby(['year','Publisher']).sum()[['count','Total_sales']]
publisher.loc[publisher['count']>=200, 'releases']='Too many releases'
publisher.loc[(publisher['count']>=100)&(publisher['count']<200), 'releases']='Many releases'
publisher.loc[publisher['count']<100, 'releases']='Few releases'
publisher = publisher.reset_index()
#publisher.sort_values(by='count',ascending=False)
fig16 = px.scatter(publisher, x="year", y="Total_sales", color='releases', hover_data=['Publisher'], size='count')
fig16.update_layout(title="Releases and total sales by publisher, 2000-2010")
py.iplot(fig16)

In [88]:
#Save image
fig16.write_image("images/fig16.png")

By taking a closer look at the graph, we can see that Microsoft is an interesting case: it has published many video games (depicted in green color) but with very low revenues. Nintendo, on the other hand, shows the greatest performance: in the year 2006, with a medium size of released video games, Nintendo had one of the largest revenues in the last 19 years. Let's take a dive into the video games published by Nintendo in 2006.

In [87]:
#Get video games released by Nintendo in 2006, by genre and by total sale
nintendo = games[(games['Publisher']=='Nintendo')&(games['year']==2006)]
nintendogames = nintendo.groupby('Genre').sum()[['count','Total_sales']].sort_values(by='Total_sales', ascending=False)
totsaleval = round(nintendogames['Total_sales'],2)

fig17 = go.Figure(data=[go.Table(header=dict(values=['<b>Genre<b>','<b>Number of video games in 2006<b>', 
                                                   '<b>Total sales in 2006 (million USD)<b>']),
                 cells=dict(values=[nintendogames.index, nintendogames['count'], totsaleval]))
                     ])
py.iplot(fig17)

Sports was the most profitable genre for Nintendo in that year (the same year of 2006 World Cup).

## Conclusions

1. There is a steady rise of number of video games released between 1980 to 2010. After 2010, the trend has been unclear. This might be a result of the **recent development of new online web-based and mobile digital platforms, such as Steam that may become strong competitors** for traditional publishers and consoles in the industry.


2. The gaming industry is extremely dynamic. In the last 10 years the composition of the industry by genres have completely changed with the rise of new genres, such as visual+novel, party, and music. Nonetheless, **sports has continuously appeared as one of the top 3 most profitable genres throughout the years.**


3. Indeed, **the gaming industry is becoming more profitable.** Total sales in 2010s are 33 times larger than total sales in 1980s. PC games might not be as profitable as other brands, but it compensates this gap by releasing vast amounts of video games.


4. PC games are more popular in PAL region. Also, in recent years, Play Station has gained terrain in PAL region with its console PS4. On the other hand, Nintendo and Xbox have not been able to expand their market beyond North America (US, Canada, Mexico) as opposed to Play Station.


5. **Innovation is what drives the business, and not numbers.** Between 2007 and 2011, publishers flooded the market with vast amounts of video games (specially PC publishers that attempted to gain terrain in the industry). However, this did not translate into higher profits. In 2006, on the other hand, Nintendo registered the highest profit of the past 2 decades. In this year, sport games became extremely profitable (perhaps due to the World Cup in Germany) and Wii was introduced into the market.

## References

* Data scrapped from VGChartz website on November 30th, 2019.
* Zion Market Research, "Cloud Gaming Market by Cloud Type (Public, Private, and Hybrid), by Streaming Type (Video and File), and by Device (Smart Phones, Tablets, Gaming Consoles, and PCs): Global Industry Perspective, Comprehensive Analysis, and Forecast, 2018—2026", available in: https://www.zionmarketresearch.com/report/cloud-gaming-market 